# Accessing FAO ASIS data

In this exercise we download the Vegetation Health Index (VHI) and Agricultural Stress Index (ASI) for an administrative region and identify the specific areas most impacted by drought.

More information on ASIS: https://www.fao.org/giews/earthobservation/index.jsp?lang=en

In [ ]:
!pip install ipykernel ipyleaflet geopandas geojson --quiet

The following cell ensures a correct download of the data from Google Cloud Storage.

In [ ]:
import os
os.environ['GS_NO_SIGN_REQUEST'] = 'YES'

In [ ]:
# Available data in ASIS:

from vito_agri_tutorials.fao.asis import ASIS_SUPPORTED_VARIABLES
import pandas as pd

df_vars = pd.DataFrame.from_dict(ASIS_SUPPORTED_VARIABLES, orient='index', columns=['Description'])
df_vars

In [ ]:
# Using this widget, you can select an administrave region from a list of available regions in a country.
# Select a country first.
# Then select a state from the list of states in the selected country.
# Hit the "Add to selection" button to add the selected state to your selection.

# Select one or two states from one country and add them to your selection.

from vito_agri_tutorials.utils.map import RegionPicker

region_picker = RegionPicker()

In [ ]:
# In this cell we extract the geometries from the selected states.

gdf = region_picker.selection_to_gdf(level=1)
gdf

In [ ]:
# We extract the timeseries of dekadal ASI data for the selected states.

from pathlib import Path
from vito_agri_tutorials.fao.asis import extract_asis_data

out_dir = Path('./results/asis')

variable = "ASI-D"
start_date = '2022-01-01'
end_date = '2024-12-31'
# We need to specify a season (1 or 2) and a landcover type (C(ropland) or G(rassland)) for the ASI data.
season = 1
landcover = "C"
write_tif = True

asi = extract_asis_data(
    variable,
    gdf,
    out_dir,
    start_date=start_date,
    end_date=end_date,
    season=season,
    landcover=landcover,
    write_tif=write_tif,
    debug=True,)

In [ ]:
# The .tif files were automatically downloaded to your out_dir. You can inspect them in QGIS.
# Now, we compute area statistics for the selected states and plot the average values across the entire states.

from vito_agri_tutorials.fao.asis import compute_area_stats, plot_stats

asi_stats = compute_area_stats(asi)
asi_stats = plot_stats(asi_stats, variable, "mean")
asi_stats

In [ ]:
# We extract the timeseries of dekadal VHI index values for the selected states.

from pathlib import Path
from vito_agri_tutorials.fao.asis import extract_asis_data

out_dir = Path('./results/asis')

variable = "VHI-D"
start_date = '2022-01-01'
end_date = '2024-12-31'
# No need to specify season or landcover for the VHI data.
season = None
landcover = None
write_tif = True

vhi = extract_asis_data(
    variable,
    gdf,
    out_dir,
    start_date=start_date,
    end_date=end_date,
    season=season,
    landcover=landcover,
    write_tif=write_tif,
    debug=True,)

In [ ]:
# Again, we compute the mean statistics of VHI over the entire region.

from vito_agri_tutorials.fao.asis import compute_area_stats, plot_stats

vhi_stats = compute_area_stats(vhi)
vhi_stats = plot_stats(vhi_stats, variable, "mean")
vhi_stats

In [ ]:
# Plot the VHI and ASI time series on the same figure for one of the selected states.

import matplotlib.pyplot as plt

state = "Southern"

# join the two relevant columns of a dataframe
df_vhi_asi = vhi_stats[[state]].join(asi_stats[[state]], lsuffix='_VHI', rsuffix='_ASI')
# rename the columns
df_vhi_asi.columns = ['mean_VHI', 'mean_ASI']

# Plot both variables on same figure
figs, ax1 = plt.subplots(1, figsize=(12, 6))
df_vhi_asi['mean_VHI'].plot(ax=ax1, label='VHI',color='blue')
ax2 = ax1.twinx()
df_vhi_asi['mean_ASI'].plot(ax=ax2, label='ASI',color="orange")

ax1.set_ylabel('Vegetation Health Index [-]')
ax1.legend(loc='upper left', ncol=5)
ax1.set_title('Comparison VHI & ASI time series')

# Set limits for both axes
ax2.set_ylabel('Agricultural Stress Index [%]')
ax2.legend(loc='upper right', ncol=5)

In [ ]:
# Download MVHI accumulated over entire growing season (MVHI-A)
# --> this can be used to check which individual pixels have suffered the most from the drought...

from pathlib import Path
from vito_agri_tutorials.fao.asis import extract_asis_data

out_dir = Path('./results/asis')

variable = "MVHI-A"
start_date = '2022-01-01'
end_date = '2024-12-31'
season = 1
landcover = "C"
sub_variable = None
write_tif = True


mvhi = extract_asis_data(variable,
                         gdf,
    out_dir,
    start_date=start_date,
    end_date=end_date,
    season=season,
    landcover=landcover,
    sub_variable=sub_variable,
    write_tif=write_tif,
    debug=True,)

In [ ]:
# To get an idea how many people are directly affected by the drought, we can use the WPOP database.
from vito_agri_tutorials.fao.wpop import WPOP_SUPPORTED_VARIABLES
import pandas as pd

df_vars = pd.DataFrame.from_dict(WPOP_SUPPORTED_VARIABLES)
df_vars = df_vars.transpose()
df_vars

In [ ]:
# Download population count data for the selected states.

from pathlib import Path
from vito_agri_tutorials.fao.wpop import extract_wpop_data

variable = "WPOP-T"

out_dir = Path('./results/wpop')

# WPOP data only available between 2000 and 2020
start_date = '2020-01-01'

wpop = extract_wpop_data(
    variable,
    gdf,
    out_dir,
    start_date,
    write_tif=True,
    debug=True,
)

In [ ]:
# Compute total number of people in the state:

from vito_agri_tutorials.fao.wpop import compute_area_stats

wpop_stats = compute_area_stats(wpop)
wpop_stats